In [ ]:
# Code taken from https://github.com/ilya-shenbin/RecVAE/blob/master/run.py

In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sc
from bottleneck import argpartition
import sys
import timeit, time

import torch
from torch import optim

import random
from copy import deepcopy

from utils import get_data, ndcg, recall
from model import VAE

sys.path.insert(0, '/home/pmoritz/dev/modsem')

from metrics import Recall, NDCG

/home/pmoritz/miniconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed = 1337
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [3]:
def parse_data(data: pd.DataFrame, uid_str: str, sid_str: str, nItems):
        nUsers = len(data[uid_str].unique())
        users = data[uid_str].astype('category').cat.codes.values
        items = data[sid_str].values
        vals = np.ones(len(data))
        X = sc.csr_matrix((vals, (users, items)), shape=(nUsers, nItems), dtype=np.float32)
        return X

In [4]:
!pwd

/home/pmoritz/dev/modsem/RecVAE


In [5]:
device = torch.device("cuda:0")

In [6]:
def generate(batch_size, device, data_in, data_out=None, shuffle=False, samples_perc_per_epoch=1):
    assert 0 < samples_perc_per_epoch <= 1
    
    total_samples = data_in.shape[0]
    samples_per_epoch = int(total_samples * samples_perc_per_epoch)
    
    if shuffle:
        idxlist = np.arange(total_samples)
        np.random.shuffle(idxlist)
        idxlist = idxlist[:samples_per_epoch]
    else:
        idxlist = np.arange(samples_per_epoch)
    
    for st_idx in range(0, samples_per_epoch, batch_size):
        end_idx = min(st_idx + batch_size, samples_per_epoch)
        idx = idxlist[st_idx:end_idx]

        yield Batch(device, idx, data_in, data_out)

In [7]:
class Batch:
    def __init__(self, device, idx, data_in, data_out=None):
        self._device = device
        self._idx = idx
        self._data_in = data_in
        self._data_out = data_out
    
    def get_idx(self):
        return self._idx
    
    def get_idx_to_dev(self):
        return torch.LongTensor(self.get_idx()).to(self._device)
        
    def get_ratings(self, is_out=False):
        data = self._data_out if is_out else self._data_in
        return data[self._idx]
    
    def get_ratings_to_dev(self, is_out=False):
        return torch.Tensor(
            self.get_ratings(is_out).toarray()
        ).to(self._device)

In [8]:
def evaluate(model, data_in, data_out, metrics, samples_perc_per_epoch=1, batch_size=500):
    metrics = deepcopy(metrics)
    model.eval()
    
    for m in metrics:
        m['score'] = []
    
    for batch in generate(batch_size=batch_size,
                          device=device,
                          data_in=data_in,
                          data_out=data_out,
                          samples_perc_per_epoch=samples_perc_per_epoch
                         ):
        
        ratings_in = batch.get_ratings_to_dev()
        ratings_out = batch.get_ratings(is_out=True)
    
        ratings_pred = model(ratings_in, calculate_loss=False).cpu().detach().numpy()
        
        if not (data_in is data_out):
            ratings_pred[batch.get_ratings().nonzero()] = -np.inf
            
        for m in metrics:
            m['score'].append(m['metric'](ratings_pred, ratings_out, k=m['k']))

    for m in metrics:
        m['score'] = np.concatenate(m['score']).mean()
        
    return [x['score'] for x in metrics]

In [9]:
def evaluate_custom(model, data_in, data_out, device, metrics_lst = ['Recall(k=20)', 'Recall(k=50)', 'NDCG(k=100)',], samples_perc_per_epoch=1, batch_size=500):
    model.to(device)
    model.eval()
    metric_callers = {}
    scores = {}
    max_topk = 0
    for metric in metrics_lst:
        try:
            metric_callers[metric] = eval(metric)
            max_topk = max(max_topk, int(metric.split("k=")[-1].strip(")")))
            scores[metric] = []
        except:
            raise NotImplementedError('metrics={} not implemented.'.format(metric))
    
    for batch in generate(batch_size=batch_size,
                          device=device,
                          data_in=data_in,
                          data_out=data_out,
                          samples_perc_per_epoch=samples_perc_per_epoch
                         ):
        
        ratings_in = batch.get_ratings_to_dev()
        ratings_out = batch.get_ratings(is_out=True)
    
        ratings_pred = model(ratings_in, calculate_loss=False).cpu().detach().numpy()
                
        if not (data_in is data_out):
            ratings_pred[batch.get_ratings().nonzero()] = -np.inf
            
        
        topk = argpartition(-ratings_pred, max_topk - 1, axis=1)[:, :max_topk]

        # we need to sort manually
        n_batch = ratings_out.shape[0]
        sorted_idx = (-ratings_pred)[np.arange(n_batch)[:, None], topk].argsort()
        topk = topk[np.arange(n_batch)[:, None], sorted_idx]
        
        for row in range(ratings_out.shape[0]):
            pred_items = topk[row, :]
            ratings_out_row  = ratings_out[row, :]
            _, true_items, _ = sc.find(ratings_out_row)
            
            for metric in metrics_lst:
                scores[metric].append(metric_callers[metric](pred_items, true_items))

    for metric in scores.keys():
        scores[metric] = np.mean(scores[metric])
        
    return scores

In [10]:
def timeit_dense_VAE(model_best, device, X_test_tr, n_users_test, max_k = 100):
    model_best.to(device=device)
    model_best.eval()
    
    rng = np.random.default_rng(seed=42)
    rand_batch_idx = rng.integers(low=0, high=n_users_test, size=1000)
    rand_query_idx = rng.integers(low=0, high=n_users_test, size=10)
    batch = torch.Tensor(X_test_tr[rand_batch_idx, :].toarray()).to(device) # dense batch
    
    def pred_batch():
        pred = model_best(batch, calculate_loss=False).detach()
        pred[batch != 0.] = -torch.inf 
        _, max_ind = pred.topk(k=max_k, axis=1)

    timer = timeit.Timer(pred_batch)
    n_timer, _ = timer.autorange()
    batch_time = min(timer.repeat(repeat=7, number=n_timer)) / n_timer

    times, indexes = [], []
    for rand_int in rand_query_idx:
        query = torch.Tensor(X_test_tr[rand_int, :].toarray()).to(device)

        def pred_query():
            pred = model_best(query, calculate_loss=False).detach()
            pred[query != 0.] = -torch.inf 
            _, max_ind = pred.topk(k=max_k)

        timer = timeit.Timer(pred_query)
        n_timer, _ = timer.autorange()
        query_time_run = min(timer.repeat(repeat=7, number=n_timer)) / n_timer

        times.append(query_time_run)
        indexes.append(rand_int)

    query_time = np.average(times) 
    return batch_time, query_time

In [11]:
def run(model, opts, train_data, batch_size, n_epochs, beta, gamma, dropout_rate):
    model.train()
    for epoch in range(n_epochs):
        for batch in generate(batch_size=batch_size, device=device, data_in=train_data, shuffle=True):
            ratings = batch.get_ratings_to_dev()

            for optimizer in opts:
                optimizer.zero_grad()
                
            _, loss = model(ratings, beta=beta, gamma=gamma, dropout_rate=dropout_rate)
            loss.backward()
            
            for optimizer in opts:
                optimizer.step()

### ML20M

In [14]:
train_ML20M   = pd.read_csv('/home/pmoritz/dev/modsem/data/ml-20m/train.csv')
val_tr_ML20M  = pd.read_csv('/home/pmoritz/dev/modsem/data/ml-20m/validation_tr.csv')
val_te_ML20M  = pd.read_csv('/home/pmoritz/dev/modsem/data/ml-20m/validation_te.csv')
test_tr_ML20M = pd.read_csv('/home/pmoritz/dev/modsem/data/ml-20m/test_tr.csv')
test_te_ML20M = pd.read_csv('/home/pmoritz/dev/modsem/data/ml-20m/test_te.csv')

train_data = parse_data(train_ML20M, 
                        uid_str='uid', 
                        sid_str='sid', 
                        nItems=len(train_ML20M.sid.unique()))
valid_in_data = parse_data(val_tr_ML20M, 
                           uid_str='uid', 
                           sid_str='sid', 
                           nItems=len(train_ML20M.sid.unique()))
valid_out_data = parse_data(val_te_ML20M, 
                            uid_str='uid', 
                            sid_str='sid', 
                            nItems=len(train_ML20M.sid.unique()))
test_in_data = parse_data(test_tr_ML20M, 
                          uid_str='uid', 
                          sid_str='sid', 
                          nItems=len(train_ML20M.sid.unique()))
test_out_data = parse_data(test_te_ML20M, 
                           uid_str='uid', 
                           sid_str='sid', 
                           nItems=len(train_ML20M.sid.unique()))

In [15]:
model_kwargs = {
    'hidden_dim': 600,
    'latent_dim': 200,
    'input_dim': train_data.shape[1]
}

In [16]:
metrics = [{'metric': ndcg, 'k': 100}]

best_ndcg = -np.inf
train_scores, valid_scores = [], []

model = VAE(**model_kwargs).to(device)
model_best = VAE(**model_kwargs).to(device)

learning_kwargs = {
    'model': model,
    'train_data': train_data,
    'batch_size': 500,
    'beta': None,
    'gamma': 0.005
}

learningrate=5e-4

decoder_params = set(model.decoder.parameters())
encoder_params = set(model.encoder.parameters())

optimizer_encoder = optim.Adam(encoder_params, lr=learningrate)
optimizer_decoder = optim.Adam(decoder_params, lr=learningrate)

In [17]:
n_epochs = 50
not_alternating = False
n_enc_epochs = 3
n_dec_epochs = 1

In [18]:
start = time.time()
for epoch in range(n_epochs):

    if not_alternating:
        run(opts=[optimizer_encoder, optimizer_decoder], n_epochs=1, dropout_rate=0.5, **learning_kwargs)
    else:
        run(opts=[optimizer_encoder], n_epochs=n_enc_epochs, dropout_rate=0.5, **learning_kwargs)
        model.update_prior()
        run(opts=[optimizer_decoder], n_epochs=n_dec_epochs, dropout_rate=0, **learning_kwargs)

    train_scores.append(
        evaluate(model, train_data, train_data, metrics, 0.01)[0]
    )
    valid_scores.append(
        evaluate(model, valid_in_data, valid_out_data, metrics, 1)[0]
    )
    
    if valid_scores[-1] > best_ndcg:
        best_ndcg = valid_scores[-1]
        model_best.load_state_dict(deepcopy(model.state_dict()))
        

    print(f'epoch {epoch} | valid ndcg@100: {valid_scores[-1]:.4f} | ' +
          f'best valid: {best_ndcg:.4f} | train ndcg@100: {train_scores[-1]:.4f}')
    
end = time.time()    
print(f'Training time: {(end-start) / 60:.2f}min')

epoch 0 | valid ndcg@100: 0.3302 | best valid: 0.3302 | train ndcg@100: 0.7137
epoch 1 | valid ndcg@100: 0.3962 | best valid: 0.3962 | train ndcg@100: 0.7631
epoch 2 | valid ndcg@100: 0.4132 | best valid: 0.4132 | train ndcg@100: 0.7726
epoch 3 | valid ndcg@100: 0.4226 | best valid: 0.4226 | train ndcg@100: 0.7813
epoch 4 | valid ndcg@100: 0.4279 | best valid: 0.4279 | train ndcg@100: 0.7859
epoch 5 | valid ndcg@100: 0.4326 | best valid: 0.4326 | train ndcg@100: 0.7895
epoch 6 | valid ndcg@100: 0.4342 | best valid: 0.4342 | train ndcg@100: 0.7943
epoch 7 | valid ndcg@100: 0.4372 | best valid: 0.4372 | train ndcg@100: 0.7993
epoch 8 | valid ndcg@100: 0.4391 | best valid: 0.4391 | train ndcg@100: 0.8030
epoch 9 | valid ndcg@100: 0.4402 | best valid: 0.4402 | train ndcg@100: 0.8055
epoch 10 | valid ndcg@100: 0.4413 | best valid: 0.4413 | train ndcg@100: 0.8079
epoch 11 | valid ndcg@100: 0.4420 | best valid: 0.4420 | train ndcg@100: 0.8113
epoch 12 | valid ndcg@100: 0.4427 | best valid: 0.

In [19]:
test_metrics = [{'metric': ndcg, 'k': 100}, {'metric': recall, 'k': 20}, {'metric': recall, 'k': 50}]

final_scores = evaluate(model_best, test_in_data, test_out_data, test_metrics)

for metric, score in zip(test_metrics, final_scores):
    print(f"{metric['metric'].__name__}@{metric['k']}:\t{score:.4f}")

ndcg@100:	0.4428
recall@20:	0.4142
recall@50:	0.5543


In [20]:
evaluate_custom(model_best, test_in_data, test_out_data, device=device)

{'Recall(k=20)': 0.41422593409041947,
 'Recall(k=50)': 0.554342493936054,
 'NDCG(k=100)': 0.44284833890062303}

In [21]:
timeit_dense_VAE(model_best, 
                 device=device, #this is the gpu
                 X_test_tr=test_in_data, 
                 n_users_test=test_in_data.shape[0], 
                 max_k = 100)

(0.007734691119985655, 0.00048638614479568787)

In [22]:
torch.save(model_best.state_dict(), 'ml20m_best')

### Netflix

In [27]:
train_netflix = pd.read_csv('/home/pmoritz/dev/modsem/data/netflix/train.csv')
val_tr_netflix = pd.read_csv('/home/pmoritz/dev/modsem/data/netflix/validation_tr.csv')
val_te_netflix = pd.read_csv('/home/pmoritz/dev/modsem/data/netflix/validation_te.csv')
test_tr_netflix = pd.read_csv('/home/pmoritz/dev/modsem/data/netflix/test_tr.csv')
test_te_netflix = pd.read_csv('/home/pmoritz/dev/modsem/data/netflix/test_te.csv')

train_data = parse_data(train_netflix, 
                        uid_str='uid', 
                        sid_str='sid', 
                        nItems=len(train_netflix.sid.unique()))

valid_in_data = parse_data(val_tr_netflix, 
                           uid_str='uid', 
                           sid_str='sid', 
                           nItems=len(train_netflix.sid.unique()))

valid_out_data = parse_data(val_te_netflix, 
                            uid_str='uid', 
                            sid_str='sid', 
                            nItems=len(train_netflix.sid.unique()))

test_in_data = parse_data(test_tr_netflix, 
                          uid_str='uid', 
                          sid_str='sid', 
                          nItems=len(train_netflix.sid.unique()))

test_out_data = parse_data(test_te_netflix, 
                           uid_str='uid', 
                           sid_str='sid', 
                           nItems=len(train_netflix.sid.unique()))

In [28]:
model_kwargs = {
    'hidden_dim': 600,
    'latent_dim': 200,
    'input_dim': train_data.shape[1]
}

In [29]:
metrics = [{'metric': ndcg, 'k': 100}]

best_ndcg = -np.inf
train_scores, valid_scores = [], []

model = VAE(**model_kwargs).to(device)
model_best = VAE(**model_kwargs).to(device)

learning_kwargs = {
    'model': model,
    'train_data': train_data,
    'batch_size': 500,
    'beta': None,
    'gamma': 0.0035
}

learningrate=5e-4

decoder_params = set(model.decoder.parameters())
encoder_params = set(model.encoder.parameters())

optimizer_encoder = optim.Adam(encoder_params, lr=learningrate)
optimizer_decoder = optim.Adam(decoder_params, lr=learningrate)

In [30]:
n_epochs = 50
not_alternating = False
n_enc_epochs = 3
n_dec_epochs = 1

In [31]:
start = time.time()
for epoch in range(n_epochs):

    if not_alternating:
        run(opts=[optimizer_encoder, optimizer_decoder], n_epochs=1, dropout_rate=0.5, **learning_kwargs)
    else:
        run(opts=[optimizer_encoder], n_epochs=n_enc_epochs, dropout_rate=0.5, **learning_kwargs)
        model.update_prior()
        run(opts=[optimizer_decoder], n_epochs=n_dec_epochs, dropout_rate=0, **learning_kwargs)

    train_scores.append(
        evaluate(model, train_data, train_data, metrics, 0.01)[0]
    )
    valid_scores.append(
        evaluate(model, valid_in_data, valid_out_data, metrics, 1)[0]
    )
    
    if valid_scores[-1] > best_ndcg:
        best_ndcg = valid_scores[-1]
        model_best.load_state_dict(deepcopy(model.state_dict()))
        

    print(f'epoch {epoch} | valid ndcg@100: {valid_scores[-1]:.4f} | ' +
          f'best valid: {best_ndcg:.4f} | train ndcg@100: {train_scores[-1]:.4f}')
    
end = time.time()    
print(f'Training time: {(end-start) / 60:.2f}min')

epoch 0 | valid ndcg@100: 0.3057 | best valid: 0.3057 | train ndcg@100: 0.6948
epoch 1 | valid ndcg@100: 0.3627 | best valid: 0.3627 | train ndcg@100: 0.7410
epoch 2 | valid ndcg@100: 0.3750 | best valid: 0.3750 | train ndcg@100: 0.7548
epoch 3 | valid ndcg@100: 0.3804 | best valid: 0.3804 | train ndcg@100: 0.7608
epoch 4 | valid ndcg@100: 0.3830 | best valid: 0.3830 | train ndcg@100: 0.7660
epoch 5 | valid ndcg@100: 0.3851 | best valid: 0.3851 | train ndcg@100: 0.7727
epoch 6 | valid ndcg@100: 0.3868 | best valid: 0.3868 | train ndcg@100: 0.7760
epoch 7 | valid ndcg@100: 0.3877 | best valid: 0.3877 | train ndcg@100: 0.7798
epoch 8 | valid ndcg@100: 0.3884 | best valid: 0.3884 | train ndcg@100: 0.7828
epoch 9 | valid ndcg@100: 0.3896 | best valid: 0.3896 | train ndcg@100: 0.7852
epoch 10 | valid ndcg@100: 0.3898 | best valid: 0.3898 | train ndcg@100: 0.7887
epoch 11 | valid ndcg@100: 0.3905 | best valid: 0.3905 | train ndcg@100: 0.7897
epoch 12 | valid ndcg@100: 0.3908 | best valid: 0.

In [32]:
test_metrics = [{'metric': ndcg, 'k': 100}, {'metric': recall, 'k': 20}, {'metric': recall, 'k': 50}]

final_scores = evaluate(model_best, test_in_data, test_out_data, test_metrics)

for metric, score in zip(test_metrics, final_scores):
    print(f"{metric['metric'].__name__}@{metric['k']}:\t{score:.4f}")

ndcg@100:	0.3948
recall@20:	0.3627
recall@50:	0.4533


In [33]:
evaluate_custom(model_best, test_in_data, test_out_data, device=device)

{'Recall(k=20)': 0.36266024174140277,
 'Recall(k=50)': 0.4533128744640806,
 'NDCG(k=100)': 0.39479793227199245}

In [34]:
timeit_dense_VAE(model_best, 
                 device=device, #this is the gpu
                 X_test_tr=test_in_data, 
                 n_users_test=test_in_data.shape[0], 
                 max_k = 100)

(0.007009308699925896, 0.0004477705160010373)

In [35]:
torch.save(model_best.state_dict(), 'netflix_best')

### MSD

In [12]:
train_msd = pd.read_csv('/home/pmoritz/dev/modsem/data/msd/train.csv')
val_tr_msd = pd.read_csv('/home/pmoritz/dev/modsem/data/msd/validation_tr.csv')
val_te_msd = pd.read_csv('/home/pmoritz/dev/modsem/data/msd/validation_te.csv')
test_tr_msd = pd.read_csv('/home/pmoritz/dev/modsem/data/msd/test_tr.csv')
test_te_msd = pd.read_csv('/home/pmoritz/dev/modsem/data/msd/test_te.csv')

train_data = parse_data(train_msd, 
                        uid_str='uid', 
                        sid_str='sid', 
                        nItems=len(train_msd.sid.unique()))

valid_in_data = parse_data(val_tr_msd, 
                           uid_str='uid', 
                           sid_str='sid', 
                           nItems=len(train_msd.sid.unique()))

valid_out_data = parse_data(val_te_msd, 
                            uid_str='uid', 
                            sid_str='sid', 
                            nItems=len(train_msd.sid.unique()))

test_in_data = parse_data(test_tr_msd, 
                          uid_str='uid', 
                          sid_str='sid', 
                          nItems=len(train_msd.sid.unique()))

test_out_data = parse_data(test_te_msd, 
                           uid_str='uid', 
                           sid_str='sid', 
                           nItems=len(train_msd.sid.unique()))

In [13]:
model_kwargs = {
    'hidden_dim': 600,
    'latent_dim': 200,
    'input_dim': train_data.shape[1]
}

In [14]:
metrics = [{'metric': ndcg, 'k': 100}]

best_ndcg = -np.inf
train_scores, valid_scores = [], []

model = VAE(**model_kwargs).to(device)
model_best = VAE(**model_kwargs).to(device)

learning_kwargs = {
    'model': model,
    'train_data': train_data,
    'batch_size': 500,
    'beta': None,
    'gamma': 0.01
}

learningrate=5e-4

decoder_params = set(model.decoder.parameters())
encoder_params = set(model.encoder.parameters())

optimizer_encoder = optim.Adam(encoder_params, lr=learningrate)
optimizer_decoder = optim.Adam(decoder_params, lr=learningrate)

In [15]:
n_epochs = 100
not_alternating = False
n_enc_epochs = 3
n_dec_epochs = 1

In [16]:
start = time.time()
for epoch in range(n_epochs):

    if not_alternating:
        run(opts=[optimizer_encoder, optimizer_decoder], n_epochs=1, dropout_rate=0.5, **learning_kwargs)
    else:
        run(opts=[optimizer_encoder], n_epochs=n_enc_epochs, dropout_rate=0.5, **learning_kwargs)
        model.update_prior()
        run(opts=[optimizer_decoder], n_epochs=n_dec_epochs, dropout_rate=0, **learning_kwargs)

    train_scores.append(
        evaluate(model, train_data, train_data, metrics, 0.01)[0]
    )
    valid_scores.append(
        evaluate(model, valid_in_data, valid_out_data, metrics, 1)[0]
    )
    
    if valid_scores[-1] > best_ndcg:
        best_ndcg = valid_scores[-1]
        model_best.load_state_dict(deepcopy(model.state_dict()))
        

    print(f'epoch {epoch} | valid ndcg@100: {valid_scores[-1]:.4f} | ' +
          f'best valid: {best_ndcg:.4f} | train ndcg@100: {train_scores[-1]:.4f}')
    
end = time.time()    
print(f'Training time: {(end-start) / 60:.2f}min')

epoch 0 | valid ndcg@100: 0.1130 | best valid: 0.1130 | train ndcg@100: 0.2387
epoch 1 | valid ndcg@100: 0.2672 | best valid: 0.2672 | train ndcg@100: 0.5032
epoch 2 | valid ndcg@100: 0.2923 | best valid: 0.2923 | train ndcg@100: 0.5589
epoch 3 | valid ndcg@100: 0.3033 | best valid: 0.3033 | train ndcg@100: 0.5875
epoch 4 | valid ndcg@100: 0.3086 | best valid: 0.3086 | train ndcg@100: 0.6045
epoch 5 | valid ndcg@100: 0.3127 | best valid: 0.3127 | train ndcg@100: 0.6170
epoch 6 | valid ndcg@100: 0.3155 | best valid: 0.3155 | train ndcg@100: 0.6283
epoch 7 | valid ndcg@100: 0.3176 | best valid: 0.3176 | train ndcg@100: 0.6379
epoch 8 | valid ndcg@100: 0.3191 | best valid: 0.3191 | train ndcg@100: 0.6455
epoch 9 | valid ndcg@100: 0.3205 | best valid: 0.3205 | train ndcg@100: 0.6520
epoch 10 | valid ndcg@100: 0.3211 | best valid: 0.3211 | train ndcg@100: 0.6567
epoch 11 | valid ndcg@100: 0.3218 | best valid: 0.3218 | train ndcg@100: 0.6614
epoch 12 | valid ndcg@100: 0.3225 | best valid: 0.

In [17]:
test_metrics = [{'metric': ndcg, 'k': 100}, {'metric': recall, 'k': 20}, {'metric': recall, 'k': 50}]

final_scores = evaluate(model_best, test_in_data, test_out_data, test_metrics)

for metric, score in zip(test_metrics, final_scores):
    print(f"{metric['metric'].__name__}@{metric['k']}:\t{score:.4f}")

ndcg@100:	0.3271
recall@20:	0.2771
recall@50:	0.3754


In [18]:
evaluate_custom(model_best, test_in_data, test_out_data, device=device)

{'Recall(k=20)': 0.27711796495266,
 'Recall(k=50)': 0.37538190093131724,
 'NDCG(k=100)': 0.32709714843412047}

In [19]:
timeit_dense_VAE(model_best, 
                 device=device, #this is the gpu
                 X_test_tr=test_in_data, 
                 n_users_test=test_in_data.shape[0], 
                 max_k = 100)

(0.025336070849880342, 0.0005171066488008364)

In [20]:
torch.save(model_best.state_dict(), 'msd_best')